In [4]:
# 本节介绍另一种常用的门控循环神经网络 LSTM （Long Short Term Memory）

# 长短期记忆 引入了三个门 输入门（input gate） 输出门（output gate） 遗忘门（forget gate）
# 以及与隐藏状态形状相同的记忆细胞

# 输入门 遗忘门 和输出门
# 三个门的输入均为当前时间步的输入以及上一个时间步的隐藏状态，并且均由激活函数为Sigmoid
# 的全连接层连接计算得到输出，即三个门元素的值域均为【0,1】
# It Ft Ot  分别有权重参数和偏差参数
# 候选记忆细胞Ct
# 可以通过元素值域在【0,1】的三个门来控制隐藏状态中的信息流动，按照元素乘法来实现的
# 记忆细胞的计算组合了上一个时间步的记忆细胞和当前时间步的候选记忆细胞，通过
# 遗忘门和输入门来控制信息的流动，从而可以有效缓解梯度衰减的问题

# 隐藏状态 Ht
# 有了记忆细胞后还可以通过输出门来控制从记忆细胞到隐藏状态Ht的信息的流动


# 读取数据集
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()


# 从零开始实现
num_inputs,num_hiddens,num_outputs = vocab_size,256,vocab_size
print('will use',device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    
    def _three():
        return (_one((num_inputs,num_hiddens)),
               _one((num_hiddens,num_hiddens)),
               torch.nn.Parameter(torch.zeros(num_hiddens,device=device,dtype=torch.float32),requires_grad=True))
    
    W_xi,W_hi,b_i = _three() # 输入门参数
    W_xf,W_hf,b_f = _three() # 遗忘门参数
    W_xo,W_ho,b_o = _three() # 输出门参数
    W_xc,W_hc,b_c = _three() # 候选记忆细胞参数
    
    # 输出层参数 输出层参数
    W_hq = _one((num_hiddens,num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs,device=device,dtype=torch.float32),requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])

# 定义模型
# 初始化函数（隐藏状态和记忆细胞）
def init_lstm_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens)),
           torch.zeros((batch_size,num_hiddens)))

def lstm(inputs,state,params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H,C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X,W_xi) + torch.matmul(H,W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X,W_xf) + torch.matmul(H,W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X,W_xo) + torch.matmul(H,W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X,W_xc) + torch.matmul(H,W_hc) + b_c)
        C = F*C + I*C_tilda
        H = O*C.tanh()
        Y = torch.matmul(H,W_hq) + b_q
        outputs.append(Y)
    return outputs,(H,C)

# 训练模型并创作歌词
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

will use cpu
epoch 40, perplexity 210.271844, time 3.51 sec
 - 分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
 - 不分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
epoch 80, perplexity 64.512702, time 3.51 sec
 - 分开 我想你这你 我不要这想 我不要这样我 不知不觉 我不要这生我 不知不觉 我不要这生活 我不要这生活
 - 不分开 你想我 你不我 想不我 别不我 我不要 我不我 你不我 你不了我不多 不知不觉 我不要这生活 我不
epoch 120, perplexity 17.074380, time 3.91 sec
 - 分开 我想你这样睡着一样 我想能这已很很 不要再再样堡 我想要这样活 我想要你 我不的我 爱不了 我想好
 - 不分开 我想要这生活 我想要你 我不的我 爱不走 是你了这了我 不知不觉 我该了这生奏 后知后觉 你已了离
epoch 160, perplexity 4.681358, time 3.48 sec
 - 分开 我想带你的微笑每天都能看到  我知道这里很美但家乡的你更美 你着我我想很很你 我要和的汉堡 我知要
 - 不分开 我想要这生活 每天依依不舍 连着了我 你过经 我想开开样 有知怎人 你知了好生活 后知不觉 我该好


In [ ]:
# 简洁实现
lr = 1e-2
lstm_layer = nn.LSTM(input_size = vocab_size,hidden_size = num_hiddens)
model = d2l.RNNModel(lstm_layer,vocab_size)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)